In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

import nltk
import regex as re
import string

In [ ]:
#/content/drive/MyDrive/SMS_train.csv encoding='cp1252'
#/content/drive/MyDrive/Sentiment.csv
#/content/drive/MyDrive/news_articles.csv
df = pd.read_csv('/content/drive/MyDrive/SMS_train.csv', encoding='cp1252')
df.head()

,S. No.,Message_body,Label
0,1,Rofl. Its true to its name,Non-Spam
1,2,The guy did some bitching but I acted like i'd...,Non-Spam
2,3,"Pity, * was in mood for that. So...any other s...",Non-Spam
3,4,Will ü b going to esplanade fr home?,Non-Spam
4,5,This is the 2nd time we have tried 2 contact u...,Spam


In [ ]:
df = df[["Message_body","Label"]]
df = df[:500]

In [ ]:
df.rename(columns={'Message_body': 'Message', 'Label': 'Category'}, inplace=True)

In [ ]:
df['Message']= df['Message'].str.replace(r'[^A-Za-z0-9 .]+', ' ')
df.head()

<ipython-input-23-51b1a7712d23>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Message']= df['Message'].str.replace(r'[^A-Za-z0-9 .]+', ' ')


,Message,Category
0,Rofl. Its true to its name,Non-Spam
1,The guy did some bitching but I acted like i d...,Non-Spam
2,Pity was in mood for that. So...any other s...,Non-Spam
3,Will b going to esplanade fr home,Non-Spam
4,This is the 2nd time we have tried 2 contact u...,Spam


In [ ]:
df.to_csv('/content/drive/MyDrive/sms_v2.csv', header=None,index = False)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Message   500 non-null    object
 1   Category  500 non-null    object
dtypes: object(2)
memory usage: 7.9+ KB


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")

In [ ]:
def get_response(text,num_return_sequences):

  encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"], encoding["attention_mask"]

  outputs = model.generate(input_ids=input_ids, attention_mask=attention_masks,
    max_length=256,
    do_sample=True,
    top_k=120,
    top_p=0.95,
    early_stopping=True,
    num_return_sequences=num_return_sequences
)
  for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
  return line

In [ ]:
def generator_function(train_df):
    train = train_df
    train = train[['Message', 'Category']]
    train['Message'] = train['Message'].apply(get_response, num_return_sequences=2)
    generated = train.explode('Message')
    generated = generated.dropna()
    generated = generated.drop_duplicates()
    return generated

In [ ]:
df_aug=generator_function(df)
df_aug.to_csv('/content/drive/MyDrive/EnsembleLearningProject/sms_aug_t5_2.csv', header=None,index = False)

Its true to its name '' Rofl ''.
True to its name it is true to Rofl.
The guy did a bitching, but I acted like i was interested in buying something else next week and he gave it to us for free.
The guy did some bitching but I acted like i would be interested in buying something else next week and he gave it to us for free.
Pity was in mood for that ; So...Any other suggestions?
Pity was in mood for that. So...any other suggestions?
Will b go to the Esplanade fr home
Will b be going to esplanade fr home.
It is the second time that we have tried 2 contact u. U have won the 750 pound prize. 2 claims are easy call 087187272008 NOW1 Only 10p per minute. BT National rate
This is the 2nd time we try 2 contact u. U have won the prize of 750 pounds. 2 claim is easy call 087187272008 NOW1 Only 10p per minute BT national rate.
To get 2.50 pounds free calling credit and details of great offers pls reply 2 this text with your valid name house no and postcode to get 2 :
To get free Call Credit and d